In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.base import TransformerMixin

In [3]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date', 'start_date_short', 'end_date_short'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['start_station','end_station','Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['start_station'] = bikeshare_machine['start_station'].astype('category')
    bikeshare_machine['end_station'] = bikeshare_machine['end_station'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['start_station','end_station','member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    included_cols2 = ['start_station','end_station','member_type','time_diff','season','month','holiday',
                     'weekday','work_day','weather_cat','temp','hum','wind','miles','rush_hour','metro_dist']
    bikeshare_machine = bikeshare_machine[included_cols2]
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['member_type','holiday','work_day','rush_hour'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['start_station','end_station','season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [4]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/metro_rush.csv'
bikeshare_machine = set_data(file)

In [5]:
bikeshare_machine = bikeshare_machine.rename(columns = {'member_type_Registered':'member_type'})
bikeshare_machine.head()

,time_diff,temp,hum,wind,miles,metro_dist,member_type,holiday_1,work_day_1,rush_hour_1,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
929625,10.983,14.755849,71.0000,13.792218,1.674238,0.450642,1,0,1,0,...,0,0,1,0,0,0,0,1,0,0
92347,12.800,2.503466,61.6957,8.696332,1.314794,0.096430,1,0,1,0,...,0,0,1,0,0,0,0,0,1,0
255348,6.117,21.688349,81.0000,15.667414,0.501247,0.210402,1,0,1,1,...,0,1,0,0,0,0,0,0,1,0
78472,5.450,2.379151,53.7917,24.250650,0.549249,0.485937,1,0,1,0,...,0,0,1,0,0,0,0,1,0,0
113146,32.283,2.666979,43.6957,12.522300,2.511347,0.669037,1,0,1,0,...,0,0,0,1,0,0,0,1,0,0


In [6]:
class CustomScaler(TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
    
    def fit(self, X, y):
        self.scaler.fit(X[:, 0:5], y)
        return self
    
    def transform(self, X):
        X_head = self.scaler.transform(X[:, 0:5])
        return np.concatenate((X_head, X[:, 5:]), axis=1)

### Model 1

In [7]:
# Leave workday, drop weekdays, leave season, drop month
# Workday overlaps with workday/not workday and months overlap with seasons
remove_cols = ['work_day_1','season_1', 'season_2', 'season_3', 'season_4','time_diff']
X1 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y1 = bikeshare_machine['time_diff']

In [8]:
# Split the data into training and testing sets and check the shape
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25, random_state=17)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((914885, 292), (304962, 292), (914885,), (304962,))

In [9]:
scale = CustomScaler()
scale.fit(X1_train, y1_train)
X1_train_scale = scale.transform(X1_train)
X1_test_scale = scale.transform(X1_test)

In [10]:
model1 = LinearRegression()
model1.fit(X1_train_scale, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [11]:
pred1 = model1.predict(X1_test_scale)
model1_r = model1.score(X1_test_scale, y1_test)
model1_mse = mean_squared_error(y1_test, pred1)
model1_rmse = np.sqrt(model1_mse)
adjustedr1 = 1 - (1-model1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test_scale.shape[1]-1)

In [12]:
model1_r, adjustedr1, model1_mse, model1_rmse

(0.41605195735349798,
 0.41549229152450717,
 79.888021118326307,
 8.9380099081577615)